# HypSelector Example

This is an example use of the HypSelector utility with a grouped reactions set of model hypotheses
as generated by HypBuilder. 

In this example, we do the selection
with Nested Sampling via Gleipnir's MultiNest wrapper class, so you will need to download and build PyMultiNest and MultiNest to run this example. You also need to have HypBuilder installed in your PYTHONPATH to run this example. See the Gleipnir [README](https://github.com/LoLab-VU/Gleipnir/blob/master/README.md) for links to these dependencies.

This example was adapted from the grouped_reactions_example from HypBuilder:
https://github.com/LoLab-VU/HypBuilder/blob/master/grouped_reactions_example.csv
There is also a script version of this example in the Gleipnir examples: 
https://github.com/LoLab-VU/Gleipnir/tree/master/examples/HypSelector/grouped_reactions

The data used in this example is synthetic data generated from model_0 with the default
parameters defined in the csv file; they are the last 10 timepoints. 

In [1]:
import numpy as np
import pysb
from pysb.simulator import ScipyOdeSimulator

The HypSelector class is imported from the pysb_utilities module:

In [2]:
from gleipnir.pysb_utilities import HypSelector

Now we can set the HypBuilder syntax model csv file:

In [3]:
# The HypBuilder format model csv file.
model_csv = 'grouped_reactions.csv'

Next we need to load the data that we want to use in the likelihood function:

In [4]:
# Load the data.
data = np.load("model_0_AB_complex_data.npy")
# The timespan of the simulations.
tspan = np.linspace(0, 5, 20)
# Define the fancy indexer or mask for the time points that the data
# corresponds to. -- In this case it is the last ten (out of 20) time points.
data_time_idxs = np.array(list(range(len(tspan))))[10:]

In this case, the data is a synthetic dataset composed of the timeseries output of the AB complex from model_0. Here is what all the model outputs look like (with the points used for calibration in red):

![Model outputs](model_outputs_fitting-data.png)

We also need to prepare the observable data for HypSelector. The data should be packaged in a dictionary keyed to the name of the observable (as defined, or to be defined in our case, in the pysb model files). The value should be 3 element tuple of the form: (data, data_sd, time_idxs). 

In [5]:
# Generate the observable data tuple for this observable: (data, data_sd, data_time_idxs)
obs_data_t = tuple((data,None,data_time_idxs))
# Generate the dictionary of observable data that is to be used in
# computing the likelihood. -- Here we are just using the AB_complex
# observable, which is the amount of A(B=1)%B(A=1).
observable_data = dict()
observable_data['AB_complex'] = obs_data_t

Now we can initialize the HypSelector with model csv file. Internally, the HypSelector instance will pass the model to HypBuilder which builds all the model variants. The created models will be in a new directory/folder named 'hb_models' and each model will be named in the format 'model_idx'.

In [6]:
# Build the HypSelector.
selector = HypSelector(model_csv)

['./output/grouped_reactions/model_1.py', './output/grouped_reactions/model_2.py', './output/grouped_reactions/model_0.py']
['hb_models/model_1.py', 'hb_models/model_2.py', 'hb_models/model_0.py']


We can check the number of models that were generated. In this case, we should have 3 model variants. 

In [7]:
# Check the number of models that were generated.
n_models = selector.number_of_models()
print("Generated {} models from input csv".format(n_models))

Generated 3 models from input csv


We need to add the following line to each of the models so that the appropriate observable is defined. This can also be done in the via the HypBuilder syntax in the csv file.

In [8]:
# Append the needed observable to the model files
obs_line = "Observable(\'AB_complex\',A(B=1)%B(A=1))"
selector.append_to_models(obs_line)

Next we need to call the gen_nested_samplers function to have the HypSelector instance construct the Nested Sampler instances for each of the models. 

In [9]:
selector.gen_nested_samplers(tspan, observable_data,
                             solver=ScipyOdeSimulator,
                             ns_version='multinest',
                             ns_population_size=100,
                             log_likelihood_type='mse')

multinest


We are now ready to run. The output is a sorted pandas DataFrame with the natural logarithm of the evidences.

In [10]:
sorted_log_evidences = selector.run_nested_sampling()

  analysing data from multinest_run_model_0_.txt
  analysing data from multinest_run_model_1_.txt
  analysing data from multinest_run_model_2_.txt


The ouput is a sorted pandas DataFrame of the log_evidence, its error estimate and the model names; the output is sorted by the log_evidence values so that the frame indices are the rankings (lowest index => highest ranked), and the first row in the frame has the largest evidence value and the highest ranked model.

In [11]:
print("The models and their log_evidence values-sorted:")
print(sorted_log_evidences)

The models and their log_evidence values-sorted:
   log_evidence  log_evidence_error    model
0     -9.189246            0.280738  model_0
1   -156.475191            0.411237  model_2
2   -960.283600            0.237607  model_1


## Bayes Factors

It's also a common practice in model selection applications to compare two models by computing the [Bayes factor](https://en.wikipedia.org/wiki/Bayes_factor) for one model over the other. From Nested Sampling, we can directly estimate the Bayes factor for two models by computing the quotient of their respective evidences; note that this assumes there is no a priori reason to prefer one model to another (i.e., each model's prior probability is the same). From the HypSelectro instance we can call the bayes_factor function: 

In [12]:
bayes_factors = selector.bayes_factors()

This function returns a cross correlation-like pandas DataFrame of the Bayes for each model comparison. Each element is given by evidence_model_column/evidence_model_row:

In [13]:
print("DataFrame of the Bayes factor matrix:")
print(bayes_factors)

DataFrame of the Bayes factor matrix:
              model_0  model_1       model_2
model_0  1.000000e+00      0.0  1.082746e-64
model_1           inf      1.0           inf
model_2  9.235777e+63      0.0  1.000000e+00


In [14]:
print("Bayes factors for the model_0 numerator ratios:")
print(bayes_factors['model_0'])

Bayes factors for the model_0 numerator ratios:
model_0    1.000000e+00
model_1             inf
model_2    9.235777e+63
Name: model_0, dtype: float64


## Akaike Information Criterion

The [Akaike Information Criterion]() (AIC) is another commonly used metric for model selection. HypSelector provides a function that returns estimates of the AIC for each model, which can used if you are interested in comparing the model ranking by evidence to that by AIC value. From the HypSelector instance call the akaike_ic function:

In [15]:
sorted_aic = selector.akaike_ic()

The function returns a sorted pandas DataFrame that is similar to the sorted_log_likelihoods returned from the run_nested_sampling function, but is sorted in ascending order by AIC value; with AIC, the model with lowest AIC value is the highest ranked. 

In [16]:
print("The models and their AIC values-sorted:")
print(sorted_aic)

The models and their AIC values-sorted:
           AIC    model
0     4.012162  model_0
1   286.738892  model_2
2  1919.154038  model_1


It's probably worth noting that in this case the AIC value for a model is estimated from the highest likelihood sample found during the Nested Sampling run. In most cases this should be fine because the highest likelihood samples collected during the Nested Sampling will typically approach the maximum likelihood value (assuming the likelihood is actually bound from above), but it is worth keeping in mind that the output values may not actually correspond to the true global maximum of the likelihood function. 